In [4]:
import string
import random
import math


def tokenize(text):
    
    """
    Input: 
        Raw text
    Output: 
        Tokenized version of text with splits on each space
    """

    split = [i for i in text.split(" ") if i != ""]
    lst = []

    for i in split:
        
        app = ""
        
        for j in range(len(i)):
            
            #Handles splitting with punctuation
            if i[j] in string.punctuation and app != "":
                lst.append(app)
                lst.append(i[j])
                app = ""

            elif i[j] in string.punctuation and app == "":
                lst.append(i[j])
                app = ""

            else:
                app += i[j]

            if j == len(i)-1 and app != "":
                lst.append(app)
    
    return(lst)


def ngrams(n, tokens):
    
    """
    Input:
        n - Number of previous tokens to include
        tokens - List of tokenized text
    Output:
        n_grams - List containing tuples with length n-1
    """
    
    n_grams = []
    including_end = tokens + ['<END>']

    for i in range(len(including_end)):

        append_lst = []
        for j in range(n-1, 0, -1):

            if i-j < 0:
                append_lst.append("<START>")

            else:
                append_lst.append(including_end[i-j])

        n_grams.append(tuple((tuple(append_lst), including_end[i])))

    return(n_grams)

In [13]:
class NgramModel(object):

    
    def __init__(self, n):
        self.n = n
        self.freqs = {}

    
    def update(self, sentence):
        
        """
        Updates the n-gram model given a new sentence
        """
        ngram = ngrams(self.n, tokenize(sentence))

        for (predecessor, successor) in ngram:

            if (predecessor, successor) not in self.freqs.keys():
                self.freqs[(predecessor, successor)] = 1

            else:   
                self.freqs[(predecessor, successor)] += 1

    
    def prob(self, context, token):
        
        """
        Input:
            context - Tuple of length n-1 giving the predecessor to the token
            token - The token whose probability is to be computed
        Output:
            probability - The conditional probability that the token occurs given the context
        """
        
        #Counts number of occurences of context followed by token
        events = sum([val for (key, val) in self.freqs.items() 
                      if key[0] == context and key[1] == token])
        
        #Counts total number of occurences of context
        total = sum([val for (key, val) in self.freqs.items() 
                     if key[0] == context])
        
        probability = events/total
        
        return(probability)

    
    def random_token(self, context):
        
        """
        Input:
            context - Tuple of length n-1 giving the predecessor to the generated string
        Output:
            Random token, based on distribution of dataset given the context
        """
        
        random_num = random.random()
        
        #Count occurrences of successors given the context
        context_occurences = sorted([(key[1], val) for (key, val) in self.freqs.items() 
                      if key[0] == context])
        choices = {x[0]:x[1] for x in context_occurences}
        
        #Counts total number of occurences of context
        total = sum([val for (key, val) in self.freqs.items() 
                     if key[0] == context])
        
        probab = 0
        
        for i in range(len(context_occurences)):

            if i == len(context_occurences) - 1:
                return(context_occurences[i][0])

            elif probab <= random_num and  random_num < probab + (choices[context_occurences[i][0]]/total):
                return(context_occurences[i][0])

            probab += choices[context_occurences[i][0]]/total


    def random_text(self, token_count):
        
        """
        Input:
            token_count - Length of the randomly generated string
        Output:
            string - Random string of length token_count using random_token
        """

        string = ""
        context = ["<START>" for i in range(self.n - 1)]
        change = 0

        for i in range(token_count):
            
            token = self.random_token(tuple(context))

            if self.n > 1:
                context = context[1:] + [token]
           
            string = string + " " + token

            if token == "<END>":
                context = ["<START>" for i in range(self.n - 1)]

        return(string)

    
    def perplexity(self, sentence):
        
        """
        Input:
            sentence - A string
        Output:
            Perplexity of the string, given the dataset
        """
        
        ngram = ngrams(self.n, tokenize(sentence))
        perp = 0

        for gram in ngram:
            perp += math.log((1/self.prob(gram[0], gram[1])))

        return((math.e ** perp)**(1/len(ngram)))

In [19]:
def create_ngram_model(n, path):
    
    """
    Input:
        n - Number of previous tokens to include
        path - File path of text
    Output:
        n_gram_obj - Object of type NgramModel trained on given text
    """
    
    file_path = open(path, "r")
    n_gram_obj = NgramModel(n)

    for line in file_path:
        n_gram_obj.update(line.replace("\n", " "))

    return(n_gram_obj)

In [31]:
m = create_ngram_model(2, "frankenstein.txt")
m.random_text(50)

' Alas , I can befall a scene is as you will not laugh in the desert that alarmed at Servox and the recess of my skin was sufficient for one of the overthrow of my undertaking or at the sun was myself , which I entered , stretched before me'

In [32]:
m = create_ngram_model(5, "frankenstein.txt")
m.random_text(50)

' At first I started back , unable to believe that it was indeed I who was reflected in the mirror ; and when I awoke , and my first care was to procure the whole works of this author , and afterwards of Paracelsus and Albertus Magnus . <END> I'

In [33]:
m = create_ngram_model(10, "frankenstein.txt")
m.random_text(50)

' But even human sympathies were not sufficient to satisfy his eager mind . <END> I sank to the ground , and my injurer , with increased swiftness , escaped into the wood . <END> Why did you confess ? <END> He began his lecture by a recapitulation of the history'

In [34]:
m = create_ngram_model(20, "frankenstein.txt")
m.random_text(50)

' I wish to soothe him , yet can I counsel one so infinitely miserable , so destitute of every hope of consolation , to live ? <END> He could not help regarding my exclamation as a presumption of my guilt and said in rather a severe tone , " I'

In [35]:
m = create_ngram_model(50, "frankenstein.txt")
m.random_text(50)

' But here also I am checked . <END> He felt as if he had been transported to fairy - land and enjoyed a happiness seldom tasted by man . <END> From you only could I hope for succour , although towards you I felt no sentiment but that of hatred'